# Notebook for generate the webscrapping of fitpass.com/mx/mapa

18/09/23 

@roman

In [1]:
import pandas as pd
import numpy as np
import requests
import re
from bs4 import BeautifulSoup
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import Select
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC

import time
from tqdm import tqdm 

Intel MKL WARNING: Support of Intel(R) Streaming SIMD Extensions 4.2 (Intel(R) SSE4.2) enabled only processors has been deprecated. Intel oneAPI Math Kernel Library 2025.0 will require Intel(R) Advanced Vector Extensions (Intel(R) AVX) instructions.
Intel MKL WARNING: Support of Intel(R) Streaming SIMD Extensions 4.2 (Intel(R) SSE4.2) enabled only processors has been deprecated. Intel oneAPI Math Kernel Library 2025.0 will require Intel(R) Advanced Vector Extensions (Intel(R) AVX) instructions.


# Part 1: Create the request

In [2]:
# URL of the website to scrape
url = "https://www.fitpass.com/mx/mapa"

# Send an HTTP GET request to the URL
response = requests.get(url)

In [3]:
print(response.text)

<!DOCTYPE html>
<html>
<head>
    <base href="https://www.fitpass.com">

    <!-- Google Tag Manager -->
    <script>(function (w, d, s, l, i) {
      w[l] = w[l] || [];
      w[l].push({
        'gtm.start':
          new Date().getTime(), event: 'gtm.js'
      });
      var f = d.getElementsByTagName(s)[0],
        j = d.createElement(s), dl = l != 'dataLayer' ? '&l=' + l : '';
      j.async = true;
      j.src =
        'https://www.googletagmanager.com/gtm.js?id=' + i + dl;
      f.parentNode.insertBefore(j, f);
    })(window, document, 'script', 'dataLayer', 'GTM-K6PVJ44');</script>
    <!-- End Google Tag Manager -->

    <!-- Google tag (gtag.js) -->
    <script async src="https://www.googletagmanager.com/gtag/js?id=G-8JJ6L118BB"></script>
    <script>
        window.dataLayer = window.dataLayer || [];
        function gtag(){dataLayer.push(arguments);}
        gtag('js', new Date());

        gtag('config', 'G-8JJ6L118BB');
    </script>

    <!-- Facebook Pixel Code -->
    <s

It seems it doesnt has the body. Before using selenium, lets try to access one studio asking the request soupb

In [6]:
soup = BeautifulSoup(response.content, "html.parser")
all_text = soup.get_text()

In [7]:
print(all_text)































Fitpass
























            ¿Cómo funciona?
          



            Membresías
          



            Studios
          



            Comunidad
          



            ¿Tienes un Studio?
          








  
  MX
  
  



      Chile
    

      Colombia
    













 



        Mapa
      





ACAPULCO
CANCUN
CDMX
CUERNAVACA
GDL
LEON
MERIDA
MONTERREY
PLAYA DEL CARMEN
PUEBLA
PUERTO VALLARTA
QUERETARO
SAN MIGUEL DE ALLENDE
TOLUCA
Ciudad




Todas
Albercas
Artes Marciales
Baile
Barre
Box
Clase Virtual (En Vivo)
Clase Virtual (IG/FB Live)
Clase Virtual (Video Grabado)
Crossfit
Cycling
Deportes
EMS
Funcional
GYM
HIIT
Pilates
Running
Wellness
Yoga
Actividad




Fitpass Pro
Fitpass Lite
Fitpass Classic
Membresía








ACAPULCO
CANCUN
CDMX
CUERNAVACA
GDL
LEON
MERIDA
MONTERREY
PLAYA DEL CARMEN
PUEBLA
PUERTO VALLARTA
QUERETARO
SAN MIGUEL DE ALLENDE
TOLUCA
Ciudad




Todas
Albercas
Artes Marciales
Baile
Barre
Box
Clase Virtual (En

No, it doesnt look it has the body. Lets try using chatgpt suggestion

In [14]:
map_content_div = soup.find("div", class_="map-content")
aside_element = map_content_div.find("aside")
important_info = aside_element.get_text()


In [15]:
important_info

'\n\n\n\n          Comienza ahora\n        \n\n'

# Part 1.2: Webscrap using selenium

In [2]:
driver = webdriver.Firefox()
url = "https://www.fitpass.com/mx/mapa"
driver.get(url)
driver.implicitly_wait(10)

In [54]:
try:
    # Find the <div> with class "list-group list-group-flush"
    list_group_div = driver.find_element(By.CLASS_NAME, "list-group.list-group-flush")
    
    if list_group_div:
        # Find all <div> elements with class "list-group-item py-3 text-start"
        list_group_items = list_group_div.find_elements(By.CSS_SELECTOR, "div.list-group-item.py-3.text-start")
        
        if list_group_items:
            list_items = []
            for item in list_group_items:
                # Get the text content of each list group item
                list_items.append(item)
        else:
            print("No list group items found within the list group div")
    else:
        print("No list group div found")
finally:
    print("ok!")

ok!


scrap via getting the html code of the page

In [58]:
print(list_items[0].get_attribute("outerHTML"))

<div class="list-group-item py-3 text-start" data-gym-name="21159 Fit Movement" data-gym-latitude="19.409432" data-gym-longitude="-99.162129" data-gym-pro="false" data-gym-notes="Centro de entrenamiento Funcional en HIIT. Somos profesionales en el área del fitness, clases de 45min. en línea y presenciales. Medidas de sanidad: cubrebocas en todo momento, toma de temperatura, gel antibacterial, cupo limitado, sana distancia y sanitización de todo el equipo." data-gym-address="Av. Monterrey 271, esquina calle aguascalientes" data-gym-virtual="false" data-gym-id="3913">
        <div class="text-muted">
          <span class="text-uppercase">
            Wellness, Crossfit, HIIT, Artes Marciales
          </span>
        </div>
        <h5 class="fw-bold my-1">
          21159 Fit Movement
        </h5>

          <div>
            Av. Monterrey 271, esquina calle aguascalientes
          </div>

        <div class="d-flex align-items-center">
            <span class="badge badge-basic me-1

In [60]:
# Close the browser window when done
driver.quit()

# Part 2: Do it in a function

In [3]:
def scrape_fitpass_gyms(url: str) -> pd.DataFrame:
    """
    Scrapes gym information from the Fitpass website and returns it as a DataFrame.
    
    Args:
        url (str): The URL of the Fitpass map page to scrape.
    
    Returns:
        pd.DataFrame: A DataFrame containing gym information.
    """
    # Create a new instance of the Firefox driver
    driver = webdriver.Firefox()

    # Open the URL in the browser
    driver.get(url)

    # Wait for some time to let the dynamic content load
    driver.implicitly_wait(10)

    # Initialize an empty dictionary to store data
    gym_data = {
        'gym_name': [],
        'latitude': [],
        'longitude': [],
        'pro_status': [],
        'notes': [],
        'address': [],
        'virtual_status': [],
        'gym_id': [],
        'activities': []
    }

    try:
        # Find the <div> with class "list-group list-group-flush"
        list_group_div = driver.find_element(By.CLASS_NAME, "list-group.list-group-flush")
        
        if list_group_div:
            # Get the inner HTML content of the list group div
            list_group_html = list_group_div.get_attribute("innerHTML")
            
            # Parse the HTML content using BeautifulSoup
            soup = BeautifulSoup(list_group_html, 'html.parser')
            
            # Find all <div> elements with class "list-group-item py-3 text-start"
            list_group_items = soup.find_all("div", class_="list-group-item py-3 text-start")
            
            if list_group_items:
                # Use tqdm to track the progress
                for item in tqdm(list_group_items, desc="Scraping Gym Data"):
                    # Access and save the desired data attributes
                    gym_data['gym_name'].append(item["data-gym-name"])
                    gym_data['latitude'].append(item["data-gym-latitude"])
                    gym_data['longitude'].append(item["data-gym-longitude"])
                    gym_data['pro_status'].append(item["data-gym-pro"])
                    gym_data['notes'].append(item["data-gym-notes"])
                    gym_data['address'].append(item["data-gym-address"])
                    gym_data['virtual_status'].append(item["data-gym-virtual"])
                    gym_data['gym_id'].append(item["data-gym-id"])
                    
                    # Find the <span> within <div class="text-muted">
                    text_muted_div = item.find("div", class_="text-muted")
                    text_uppercase_span = text_muted_div.find("span", class_="text-uppercase")
                    gym_data['activities'].append(text_uppercase_span.get_text(strip=True))
                    
            else:
                print("No list group items found within the list group div")
        else:
            print("No list group div found")
    finally:
        # Close the browser window when done
        driver.quit()

    # Create a DataFrame from the extracted dictionary
    df = pd.DataFrame(gym_data)

    return df


In [4]:
# URL of the website to scrape
fitpass_url = "https://www.fitpass.com/mx/mapa"

# Call the function to scrape and get the DataFrame
df_fitpass_mexico = scrape_fitpass_gyms(fitpass_url)


Scraping Gym Data: 100%|██████████| 863/863 [00:00<00:00, 35349.86it/s]


In [6]:
df_fitpass_mexico

,gym_name,latitude,longitude,pro_status,notes,address,virtual_status,gym_id,activities
0,21159 Fit Movement,19.409432,-99.162129,false,Centro de entrenamiento Funcional en HIIT. Som...,"Av. Monterrey 271, esquina calle aguascalientes",false,3913,"Wellness, Crossfit, HIIT, Artes Marciales"
1,321 Move!,19.359657,-99.202687,false,Movimiento y espacio virtual y presencial de F...,"Pasaje Las Águilas, Calzada de las Águilas 321...",false,4356,"Funcional, Artes Marciales, Baile, HIIT, Depor..."
2,360Cycle Virtual,21.118018,-101.656584,false,En 360CYCLE creemos que para obtener resultado...,Blvd. Adolfo López Mateos,true,3529,"Cycling, Clase Virtual (Video Grabado)"
3,3 Are Legend,19.539159,-99.182145,false,Entrenamiento Funcional y box.,Jesus Reyes Heroles 112,false,5576,"Funcional, Box"
4,40º Grados Bikram Yoga,19.552389,-99.270923,false,Estudio de Bikram Yoga y más en Zona Esmeralda,Av. Adolfo Ruiz Cortines 247,false,3735,"Yoga, HIIT"
...,...,...,...,...,...,...,...,...,...
858,Zonadanza: Ballet Studio & Art,19.390768,-99.291066,false,Escuela formativa con más de 13 años en el mer...,Avenida Jesus del Monte # 17 Piso 2,false,17,"Baile, Artes Marciales, Funcional, Yoga, Pilates"
859,Zuda Lilas,19.38801,-99.247764,true,Wellness and Fitness Studio.,Calle Paseo de Las Lilas 92,false,1623,"HIIT, Funcional, Yoga, Baile, Box, Cycling, We..."
860,Zuda Prado Norte,19.427231,-99.211101,true,Estudio de Wellness que integra tres estudios ...,"Prado Norte 420, Piso 3",false,3976,"Cycling, HIIT, Funcional, Barre, Pilates, Yoga..."
861,Zuda Virtual,19.405721,-99.252566,false,Wellness and Fitness Studio.,Paseo de las Lilas 92 - Plaza Lilas piso 1,true,3101,"Yoga, Clase Virtual (Video Grabado), Pilates, ..."


# Part 3: Scrap all mexico [BLOCKED]

In [73]:
def scrape_all_cities_fitpass_gyms(base_url: str, cities: list) -> pd.DataFrame:
    """
    Scrapes gym information from the Fitpass website for all specified cities and returns it as a DataFrame.

    Args:
        base_url (str): The base URL of the Fitpass map page.
        cities (list): A list of dictionaries containing city information (id and name).

    Returns:
        pd.DataFrame: A DataFrame containing gym information for all cities.
    """
    all_gyms_df = pd.DataFrame()  # Initialize an empty DataFrame to store all gym data

    for city in tqdm(cities, desc="Scraping Gym Data"):
        city_id = city['id']
        city_name = city['name']
        city_url = f"{base_url}?city_id={city_id}"
        
        print(f"Scraping gyms for {city_name}...")
        
        # Call the previous function to scrape gym data for the current city
        city_gyms_df = scrape_fitpass_gyms(city_url)
        
        # Add a column for the city name
        city_gyms_df['city_name'] = city_name
        
        # Append the current city's gym data to the all_gyms_df DataFrame
        all_gyms_df = pd.concat([all_gyms_df, city_gyms_df], axis=0, ignore_index=True)

        # time sleep
        sleep_time = np.random.randint(3, 8)
        print(f"Sleeping for {sleep_time} seconds...\n")
        time.sleep(sleep_time)
        
    return all_gyms_df

In [74]:
# scrap cities
cities_list = [
    {"id": 50, "name": "ACAPULCO"},
    {"id": 8, "name": "CANCUN"},
    {"id": 1, "name": "CDMX"},
    {"id": 4, "name": "CUERNAVACA"},
    {"id": 2, "name": "GDL"},
    {"id": 15, "name": "LEON"},
    {"id": 9, "name": "MERIDA"},
    {"id": 6, "name": "MONTERREY"},
    {"id": 11, "name": "PLAYA DEL CARMEN"},
    {"id": 3, "name": "PUEBLA"},
    {"id": 14, "name": "PUERTO VALLARTA"},
    {"id": 7, "name": "QUERETARO"},
    {"id": 12, "name": "SAN MIGUEL DE ALLENDE"},
    {"id": 5, "name": "TOLUCA"}
]

# Base URL of the website to scrape
fitpass_base_url = "https://www.fitpass.com/mx/mapa"

# Call the function to scrape and get the DataFrame for all cities
all_cities_gym_df = scrape_all_cities_fitpass_gyms(fitpass_base_url, cities_list)

# Save the DataFrame to a CSV file
all_cities_gym_df.to_csv('fitpass_all_cities_gyms.csv', index=False)

# Display the DataFrame
print(all_cities_gym_df)

Scraping Gym Data:   0%|          | 0/14 [00:00<?, ?it/s]

Scraping gyms for ACAPULCO...


Scraping Gym Data: 100%|██████████| 863/863 [00:00<00:00, 42019.48it/s]


Sleeping for 7 seconds...



Scraping Gym Data:   7%|▋         | 1/14 [00:14<03:02, 14.02s/it]

...

Scraping gyms for CUERNAVACA...


Scraping Gym Data: 100%|██████████| 863/863 [00:00<00:00, 48010.88it/s]


Sleeping for 5 seconds...



Scraping Gym Data:  21%|██▏       | 3/14 [00:47<02:54, 15.86s/it]


KeyboardInterrupt: 

## 3.1 Try to get the data from the website from each selection of the dropdown


In [106]:
driver = webdriver.Chrome()  # You can use other WebDriver options as well
url = "https://www.fitpass.com/mx/mapa"
driver.get(url)

In [107]:
wait = WebDriverWait(driver, 10)
option = wait.until(EC.presence_of_element_located((By.CSS_SELECTOR, f"select[name='city_id'] option[value='{city_id}']")))
driver.execute_script("arguments[0].scrollIntoView();", option)
option.click()

ElementNotInteractableException: Message: element not interactable: Element is not currently visible and may not be manipulated
  (Session info: chrome=115.0.5790.170)
Stacktrace:
0   chromedriver                        0x0000000102a32a6c chromedriver + 4303468
1   chromedriver                        0x0000000102a2b198 chromedriver + 4272536
2   chromedriver                        0x000000010265f594 chromedriver + 292244
3   chromedriver                        0x0000000102663d20 chromedriver + 310560
4   chromedriver                        0x0000000102665894 chromedriver + 317588
5   chromedriver                        0x00000001026659c0 chromedriver + 317888
6   chromedriver                        0x00000001026a0798 chromedriver + 559000
7   chromedriver                        0x00000001026931b4 chromedriver + 504244
8   chromedriver                        0x0000000102692220 chromedriver + 500256
9   chromedriver                        0x00000001026d5628 chromedriver + 775720
10  chro

In [111]:
option = driver.find_element(By.CSS_SELECTOR, f"select[name='city_id'] option[value='{city_id}']")

NoSuchElementException: Message: no such element: Unable to locate element: {"method":"css selector","selector":"select[name='city_id'] option[value='50']"}
  (Session info: chrome=115.0.5790.170); For documentation on this error, please visit: https://www.selenium.dev/documentation/webdriver/troubleshooting/errors#no-such-element-exception
Stacktrace:
0   chromedriver                        0x0000000102a32a6c chromedriver + 4303468
1   chromedriver                        0x0000000102a2b198 chromedriver + 4272536
2   chromedriver                        0x000000010265f594 chromedriver + 292244
3   chromedriver                        0x000000010269c8fc chromedriver + 542972
4   chromedriver                        0x00000001026d5628 chromedriver + 775720
5   chromedriver                        0x0000000102690b40 chromedriver + 494400
6   chromedriver                        0x0000000102691988 chromedriver + 498056
7   chromedriver                        0x00000001029f3924 chromedriver + 40

In [105]:
driver.close()

I got stucked for all the cities. I should get a little course on how to use selenium

scroll into view

# Part 3: Wrangling the data

## Part 3.1: Look whick type of data we have

In [7]:
df_fitpass_mexico.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 863 entries, 0 to 862
Data columns (total 9 columns):
 #   Column          Non-Null Count  Dtype 
---  ------          --------------  ----- 
 0   gym_name        863 non-null    object
 1   latitude        863 non-null    object
 2   longitude       863 non-null    object
 3   pro_status      863 non-null    object
 4   notes           863 non-null    object
 5   address         863 non-null    object
 6   virtual_status  863 non-null    object
 7   gym_id          863 non-null    object
 8   activities      863 non-null    object
dtypes: object(9)
memory usage: 60.8+ KB


In [32]:
df_fitpass_mexico.notes[200]

'DANCEFIT es un espacio para que las personas se reúnan y se diviertan mientras se ponen en forma. ¡Puedes bailar tu camino hacia la buena salud! Con una variedad de eventos, DANCEFIT es una forma única de mantenerse saludable y divertirse.'

In [12]:
df_fitpass_mexico

,gym_name,latitude,longitude,pro_status,notes,address,virtual_status,gym_id,activities
0,21159 Fit Movement,19.409432,-99.162129,false,Centro de entrenamiento Funcional en HIIT. Som...,"Av. Monterrey 271, esquina calle aguascalientes",false,3913,"Wellness, Crossfit, HIIT, Artes Marciales"
1,321 Move!,19.359657,-99.202687,false,Movimiento y espacio virtual y presencial de F...,"Pasaje Las Águilas, Calzada de las Águilas 321...",false,4356,"Funcional, Artes Marciales, Baile, HIIT, Depor..."
2,360Cycle Virtual,21.118018,-101.656584,false,En 360CYCLE creemos que para obtener resultado...,Blvd. Adolfo López Mateos,true,3529,"Cycling, Clase Virtual (Video Grabado)"
3,3 Are Legend,19.539159,-99.182145,false,Entrenamiento Funcional y box.,Jesus Reyes Heroles 112,false,5576,"Funcional, Box"
4,40º Grados Bikram Yoga,19.552389,-99.270923,false,Estudio de Bikram Yoga y más en Zona Esmeralda,Av. Adolfo Ruiz Cortines 247,false,3735,"Yoga, HIIT"
...,...,...,...,...,...,...,...,...,...
858,Zonadanza: Ballet Studio & Art,19.390768,-99.291066,false,Escuela formativa con más de 13 años en el mer...,Avenida Jesus del Monte # 17 Piso 2,false,17,"Baile, Artes Marciales, Funcional, Yoga, Pilates"
859,Zuda Lilas,19.38801,-99.247764,true,Wellness and Fitness Studio.,Calle Paseo de Las Lilas 92,false,1623,"HIIT, Funcional, Yoga, Baile, Box, Cycling, We..."
860,Zuda Prado Norte,19.427231,-99.211101,true,Estudio de Wellness que integra tres estudios ...,"Prado Norte 420, Piso 3",false,3976,"Cycling, HIIT, Funcional, Barre, Pilates, Yoga..."
861,Zuda Virtual,19.405721,-99.252566,false,Wellness and Fitness Studio.,Paseo de las Lilas 92 - Plaza Lilas piso 1,true,3101,"Yoga, Clase Virtual (Video Grabado), Pilates, ..."


## 3.2 Extract information using NLP techniques

In [36]:
# first, lower case all the text in 'activities' column
df_fitpass_mexico['activities'] = df_fitpass_mexico['activities'].str.lower()


,0,1,2,3,4,5,6,7,8,9,10,11
0,wellness,crossfit,hiit,artes marciales,None,None,None,None,None,None,None,None
1,funcional,artes marciales,baile,hiit,deportes,box,barre,pilates,None,None,None,None
2,cycling,clase virtual (video grabado),None,None,None,None,None,None,None,None,None,None
3,funcional,box,None,None,None,None,None,None,None,None,None,None
4,yoga,hiit,None,None,None,None,None,None,None,None,None,None
...,...,...,...,...,...,...,...,...,...,...,...,...
858,baile,artes marciales,funcional,yoga,pilates,None,None,None,None,None,None,None
859,hiit,funcional,yoga,baile,box,cycling,wellness,running,crossfit,barre,pilates,None
860,cycling,hiit,funcional,barre,pilates,yoga,baile,crossfit,None,None,None,None
861,yoga,clase virtual (video grabado),pilates,barre,cycling,hiit,funcional,crossfit,None,None,None,None


In [74]:
# second, unnest the text in 'activities' column by comma into new columns
df_fitpass_mexico_long = df_fitpass_mexico[['gym_id', 'activities']].copy().assign(
    activities=lambda dfx: dfx['activities'].str.split(',')).explode('activities')
df_fitpass_mexico_long

,gym_id,activities
0,3913,wellness
0,3913,crossfit
0,3913,hiit
0,3913,artes marciales
1,4356,funcional
...,...,...
862,5008,baile
862,5008,pilates
862,5008,box
862,5008,yoga


In [90]:
# second bis, pivot long to wide from activities
df_fitpass_mexico_wide = (
    df_fitpass_mexico_long
    .copy()
    .assign(
        activities = lambda x: 
            (x['activities'].str.strip()
             .str.replace(' ', '_')
             .str.lower()
             .str.replace(r'_\([^)]*\)', '', regex = True) # get rid from "_()" in "prefix_()"
             ),
        activities_present = 1
    )
    .replace({
        'albercas': 'pool',
        'artes_marciales': 'mma',
        'baile': 'dance',
        'barre': 'barre',
        'box': 'box',
        'clase_virtual': 'virtual_class',
        'crossfit': 'crossfit',
        'cycling': 'cycling',
        'deportes': 'sports',
        'ems': 'ems',
        'funcional': 'functional',
        'gym': 'gym',
        'hiit': 'hiit',
        'pilates': 'pilates',
        'running': 'running',
        'wellness': 'wellness',
        'yoga': 'yoga'
    })
    .drop_duplicates()
    .query('activities != ""')
    .pivot_table(
        index = 'gym_id',
        columns = 'activities',
        values = 'activities_present',
        fill_value = 0
    )
    .reset_index()
)

df_fitpass_mexico_wide

activities,gym_id,barre,box,crossfit,cycling,dance,ems,functional,gym,hiit,mma,pilates,pool,running,sports,virtual_class,wellness,yoga
0,1006,1,1,0,1,1,0,1,1,1,1,1,0,1,1,0,0,1
1,1009,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,1,0
2,1024,0,0,1,0,0,0,1,0,1,0,0,0,0,0,0,0,0
3,1031,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0
4,1036,0,0,0,0,0,1,1,0,0,0,0,0,0,0,0,1,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
854,967,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0
855,969,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0
856,978,1,0,0,0,1,0,1,0,1,0,1,0,0,0,0,0,1
857,984,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0


In [91]:
df_fitpass_mexico.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 863 entries, 0 to 862
Data columns (total 9 columns):
 #   Column          Non-Null Count  Dtype 
---  ------          --------------  ----- 
 0   gym_name        863 non-null    object
 1   latitude        863 non-null    object
 2   longitude       863 non-null    object
 3   pro_status      863 non-null    object
 4   notes           863 non-null    object
 5   address         863 non-null    object
 6   virtual_status  863 non-null    object
 7   gym_id          863 non-null    object
 8   activities      863 non-null    object
dtypes: object(9)
memory usage: 60.8+ KB


## 3.2: Get the duration of the class from the "notes" column

In [130]:
# get the number and the first string when match a number pattern in "notes" column. 
# if there are multiple numbers, get a list of the number-string pairs
# if there is no number, return None
def get_number_string_pair(x):
    if x is None:
        return None
    else:
        # get the first number-string pair
        number_string_pair = re.findall(r'(\d+)\s*(\w+)', x)
        if len(number_string_pair) > 0:
            return number_string_pair
        else:
            return []

In [131]:
# example 1
example_note_time = df_fitpass_mexico.copy().notes[110]

print("Description: \n\n\t", example_note_time)
print(f"{'='*150}")
print("regex: \n\n\t", get_number_string_pair(example_note_time))

Description: 

	 Contenedor Boutique Boxing Studio, vivirás al aire libre la experiencia Black Box y la metodología Black Box System que integra rounds de Boxeo y de ejercicios funcionales semi personalizados. Aforo máximo 8 personas. Se sanitiza todo el equipo. Gel y toma de temperatura. Uso de cubrebocas opcional para asistentes, el coach se lo pone en manopleos
regex: 

	 [('8', 'personas')]


In [132]:
df_fitpass_mexico['possible_minutes'] = df_fitpass_mexico['notes'].apply(get_number_string_pair)

In [133]:
df_fitpass_mexico.possible_minutes

0          [(45, min)]
1      [(9, formatos)]
2       [(360, CYCLE)]
3                   []
4                   []
            ...       
858       [(13, años)]
859                 []
860                 []
861                 []
862                 []
Name: possible_minutes, Length: 863, dtype: object

In [134]:
df_fitpass_mexico['num_matching'] = df_fitpass_mexico['possible_minutes'].apply(len) 

In [135]:
df_fitpass_mexico['num_matching'].value_counts()

num_matching
0    708
1     90
2     39
3     19
5      5
6      1
4      1
Name: count, dtype: int64

In [161]:
df_fitpass_mexico.query('num_matching < 1').sort_values('num_matching', ascending=False).head(10)

,gym_name,latitude,longitude,pro_status,notes,address,virtual_status,gym_id,activities,possible_minutes,num_matching,class_minutes
3,3 Are Legend,19.539159,-99.182145,false,Entrenamiento Funcional y box.,Jesus Reyes Heroles 112,false,5576,"funcional, box",[],0,30
559,Shift,19.421946,-99.159508,false,"Estudio de entrenamiento funcional, donde se t...",Córdoba 49 piso 1,false,495,"funcional, gym",[],0,30
587,Smart Fit Camarones,19.47855,-99.188263,false,Nuestra misión es entregar fitness de alta cal...,"Centenario 520, casi esquina con Eje 3 Norte (...",false,1920,gym,[],0,30
588,Smart Fit Centrum Park,19.534888,-99.207081,false,Nuestra misión es entregar fitness de alta cal...,Avenida Gustavo Baz Prada 309,false,2414,gym,[],0,30
589,Smart Fit Churubusco,19.38006,-99.101931,false,Nuestra misión es entregar fitness de alta cal...,Plaza Zentralia Churubusco. Avenida Río Churub...,false,1896,gym,[],0,30
590,Smart Fit Circuito Azteca,19.301815,-99.163034,false,Nuestra misión es entregar fitness de alta cal...,Periférico sur 5580,false,1894,gym,[],0,30
591,Smart Fit Citiinsurgentes,19.397813,-99.171462,false,Nuestra misión es entregar fitness de alta cal...,Insurgentes Sur 601,false,1891,gym,[],0,30
592,Smart Fit Clavería,19.463593,-99.181851,false,En nuestras instalaciones ofrecemos una amplia...,Egipto 142 SD - Dentro de Patio Clavería,false,5302,gym,[],0,30
593,Smart Fit Coapa,19.312764,-99.136303,false,Nuestra misión es entregar fitness de alta cal...,"Calle Benito Juárez No. 117, Ex-Hacienda Coapa",false,2453,gym,[],0,30
594,Smart Fit Condesa,19.410964,-99.182206,false,Nuestra misión es entregar fitness de alta cal...,"Alfonso Reyes No. 5, esquina José Vasconcelos",false,1890,gym,[],0,30


In [162]:
# From the tuples in 'possible_minutes' get the second element of each tuple, get rid of special spanish characters, and compare if it is equal to min, minutos, minutes
# if it is true, then return the first element of the tuple, which is the number of minutes
# if there are multiple matches, return the maximum
def get_minutes(list_of_tuples: list[tuple]):
    # case where there are not matches
    if len(list_of_tuples) == 0:
        return np.nan
    
    # case where there are matches
    list_of_minutes = list()
    for i in list_of_tuples:
        # get rid of special spanish characters
        if i[1].replace('á', 'a').replace('ó', 'o').replace('ú', 'u').replace('í', 'i') in ['min', 'minutos', 'minutes']:
            list_of_minutes.append(i[0])

    # case where there are not matches
    if len(list_of_minutes) == 0:
        return np.nan
    else:
        return max(list_of_minutes)

In [163]:
df_fitpass_mexico['class_minutes'] = df_fitpass_mexico['possible_minutes'].apply(get_minutes)

In [164]:
df_fitpass_mexico[~df_fitpass_mexico['class_minutes'].isna()]

,gym_name,latitude,longitude,pro_status,notes,address,virtual_status,gym_id,activities,possible_minutes,num_matching,class_minutes
0,21159 Fit Movement,19.409432,-99.162129,false,Centro de entrenamiento Funcional en HIIT. Som...,"Av. Monterrey 271, esquina calle aguascalientes",false,3913,"wellness, crossfit, hiit, artes marciales","[(45, min)]",1,45
6,9Round Insurgentes Mixcoac,19.366215,-99.182319,false,9Round es un concepto innovador de 9 estacione...,Cda. Perpetua #35,false,4427,box,"[(9, Round), (9, estaciones), (30, minutos), (...",5,30
7,9Round Interlomas,19.399524,-99.276491,true,Nuestro sistema es a base de 9 estaciones desa...,"Blvd. Magno Centro Lote 1 Mza 2 Local 3, Plaza...",false,4149,box,"[(9, estaciones), (30, minutos)]",2,30
8,9Round Jinetes Valle Dorado,19.551237,-99.209656,false,9Round es un sistema de entrenamiento único a ...,"Atenas 6, Local 30 PB, Valle Dorado",false,4865,box,"[(9, Round), (30, minutos), (50, 0)]",3,30
9,9Round Plaza Bosques,19.407241,-99.239487,true,9Round es un gimnasio especializado a base de ...,Bosque de Duraznos 187 - PA,false,4166,box,"[(9, Round), (30, minutos), (500, calorías)]",3,30
10,9Round Polanco,19.436536,-99.209774,true,Nuestro sistema es a base de 9 estaciones desa...,Homero 1730 Local B,false,4327,box,"[(9, estaciones), (30, minutos)]",2,30
11,9Round Rio Volga,19.427195,-99.170186,false,Nuestro sistema es a base de 9 estaciones desa...,Río Volga 89,false,4155,box,"[(9, estaciones), (30, minutos)]",2,30
12,9Round San Jeronimo,19.323893,-99.227514,false,Estudio de Box y kickboxing. Nuestro circuito ...,Av. San Jerónimo 819-Local 16,false,5624,box,"[(30, min), (500, calorías), (9, Round)]",3,30
13,9Round Santa Fe,19.357076,-99.283885,true,Nuestro sistema es a base de 9 estaciones desa...,"Centro Infinito Santa Fe, Vasco de Quiroga 4299",false,4150,box,"[(9, estaciones), (30, minutos)]",2,30
14,9Round Zona Azul,19.503996,-99.249346,false,Nuestro sistema es a base de 9 estaciones desa...,Cto. Circunvalacion Pte. 146 Local 31,false,4152,box,"[(9, estaciones), (30, minutos)]",2,30


In [165]:
# look how many nan values are there in the data
df_fitpass_mexico['class_minutes'].isna().mean()

0.9304750869061413

In [166]:
# fill values of "class_minutes" with the mode (do not)
# df_fitpass_mexico['class_minutes'] = df_fitpass_mexico['class_minutes'].fillna(df_fitpass_mexico['class_minutes'].mode()[0])

In [167]:
df_fitpass_mexico['class_minutes']

0       45
1      NaN
2      NaN
3      NaN
4      NaN
      ... 
858    NaN
859    NaN
860    NaN
861    NaN
862    NaN
Name: class_minutes, Length: 863, dtype: object

In [ ]:
# Part 4: Gather all the data and save it in a csv file


In [171]:
df_main = (
    df_fitpass_mexico
    .assign(
        pro_status = lambda x: x['pro_status'].apply(lambda x: 1 if x == 'true' else 0),
        virtual_status = lambda x: x['virtual_status'].apply(lambda x: 1 if x == 'true' else 0)
    )
    [[
        'gym_id', 'gym_name', 'pro_status', 'virtual_status',
        'class_minutes', 'notes', 
        'latitude', 'longitude', 'address'
        ]]
    )
df_main

,gym_id,gym_name,pro_status,virtual_status,class_minutes,notes,latitude,longitude,address
0,3913,21159 Fit Movement,0,0,45,Centro de entrenamiento Funcional en HIIT. Som...,19.409432,-99.162129,"Av. Monterrey 271, esquina calle aguascalientes"
1,4356,321 Move!,0,0,NaN,Movimiento y espacio virtual y presencial de F...,19.359657,-99.202687,"Pasaje Las Águilas, Calzada de las Águilas 321..."
2,3529,360Cycle Virtual,0,1,NaN,En 360CYCLE creemos que para obtener resultado...,21.118018,-101.656584,Blvd. Adolfo López Mateos
3,5576,3 Are Legend,0,0,NaN,Entrenamiento Funcional y box.,19.539159,-99.182145,Jesus Reyes Heroles 112
4,3735,40º Grados Bikram Yoga,0,0,NaN,Estudio de Bikram Yoga y más en Zona Esmeralda,19.552389,-99.270923,Av. Adolfo Ruiz Cortines 247
...,...,...,...,...,...,...,...,...,...
858,17,Zonadanza: Ballet Studio & Art,0,0,NaN,Escuela formativa con más de 13 años en el mer...,19.390768,-99.291066,Avenida Jesus del Monte # 17 Piso 2
859,1623,Zuda Lilas,1,0,NaN,Wellness and Fitness Studio.,19.38801,-99.247764,Calle Paseo de Las Lilas 92
860,3976,Zuda Prado Norte,1,0,NaN,Estudio de Wellness que integra tres estudios ...,19.427231,-99.211101,"Prado Norte 420, Piso 3"
861,3101,Zuda Virtual,0,1,NaN,Wellness and Fitness Studio.,19.405721,-99.252566,Paseo de las Lilas 92 - Plaza Lilas piso 1


In [172]:
# merge with activities
df_main = df_main.merge(
    df_fitpass_mexico_wide,
    how="left",
    on='gym_id'
    )


In [183]:
df_main

,gym_id,gym_name,pro_status,virtual_status,class_minutes,notes,latitude,longitude,address,barre,...,gym,hiit,mma,pilates,pool,running,sports,virtual_class,wellness,yoga
0,3913,21159 Fit Movement,0,0,45,Centro de entrenamiento Funcional en HIIT. Som...,19.409432,-99.162129,"Av. Monterrey 271, esquina calle aguascalientes",0.0,...,0.0,1.0,1.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0
1,4356,321 Move!,0,0,NaN,Movimiento y espacio virtual y presencial de F...,19.359657,-99.202687,"Pasaje Las Águilas, Calzada de las Águilas 321...",1.0,...,0.0,1.0,1.0,1.0,0.0,0.0,1.0,0.0,0.0,0.0
2,3529,360Cycle Virtual,0,1,NaN,En 360CYCLE creemos que para obtener resultado...,21.118018,-101.656584,Blvd. Adolfo López Mateos,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0
3,5576,3 Are Legend,0,0,NaN,Entrenamiento Funcional y box.,19.539159,-99.182145,Jesus Reyes Heroles 112,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,3735,40º Grados Bikram Yoga,0,0,NaN,Estudio de Bikram Yoga y más en Zona Esmeralda,19.552389,-99.270923,Av. Adolfo Ruiz Cortines 247,0.0,...,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
858,17,Zonadanza: Ballet Studio & Art,0,0,NaN,Escuela formativa con más de 13 años en el mer...,19.390768,-99.291066,Avenida Jesus del Monte # 17 Piso 2,0.0,...,0.0,0.0,1.0,1.0,0.0,0.0,0.0,0.0,0.0,1.0
859,1623,Zuda Lilas,1,0,NaN,Wellness and Fitness Studio.,19.38801,-99.247764,Calle Paseo de Las Lilas 92,1.0,...,0.0,1.0,0.0,1.0,0.0,1.0,0.0,0.0,1.0,1.0
860,3976,Zuda Prado Norte,1,0,NaN,Estudio de Wellness que integra tres estudios ...,19.427231,-99.211101,"Prado Norte 420, Piso 3",1.0,...,0.0,1.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,1.0
861,3101,Zuda Virtual,0,1,NaN,Wellness and Fitness Studio.,19.405721,-99.252566,Paseo de las Lilas 92 - Plaza Lilas piso 1,1.0,...,0.0,1.0,0.0,1.0,0.0,0.0,0.0,1.0,0.0,1.0


In [176]:
!pwd

/Users/ravj/Documents/study/ITAM-MASTER/sem1/fitpass-plan-optimizer/src


In [179]:
# save into a parquet file
df_main.to_parquet('../data/scrapped_fitpass_cdmx.parquet')

In [181]:
df_read = pd.read_parquet('../data/scrapped_fitpass_cdmx.parquet')